### Code for Homework Assignment #2

In [1]:
import requests
from bs4 import BeautifulSoup
from collections import deque
import re
from config import configs


In [2]:
headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    "Cookie": '_ga_DHNWEVCZW6=GS1.1.1696612350.3.0.1696612350.0.0.0; _gcl_aw=GCL.1696756177.Cj0KCQjwpompBhDZARIsAFD_Fp_e460gXyjWGBgp9K9t3jg1l5KV4qdPpK6ul88XeGs6ZGcS1Vq770kaAuuvEALw_wcB; _gcl_au=1.1.846921473.1696756177; _ga_PSXRXB3M0B=GS1.1.1696756176.1.0.1696756176.0.0.0; _gid=GA1.2.344435673.1696756177; _gac_UA-4803886-1=1.1696756177.Cj0KCQjwpompBhDZARIsAFD_Fp_e460gXyjWGBgp9K9t3jg1l5KV4qdPpK6ul88XeGs6ZGcS1Vq770kaAuuvEALw_wcB; _gat_UA-4803886-1=1; _gac_UA-151359273-10=1.1696756177.Cj0KCQjwpompBhDZARIsAFD_Fp_e460gXyjWGBgp9K9t3jg1l5KV4qdPpK6ul88XeGs6ZGcS1Vq770kaAuuvEALw_wcB; _gat_UA-151359273-10=1; pageviewCount=2; _ga_TK9BQL5X7Z=GS1.1.1696756176.4.1.1696756178.0.0.0; _ga_GYRY47YH3Z=GS1.1.1696756176.1.1.1696756178.0.0.0; _ga_Z047KJJ47B=GS1.1.1696756176.1.1.1696756178.0.0.0; AWSALBTG=oauqL7VVR3ZhtBLHvQlk2f/FwFolND75LZgKcSh9mj81rbbCtKXarFgybh8vNAjL+B5bWVbv9v/oGOG8VXtoS7WK8BEtE6A1C0vXCTi8V/WetP6nz4Nss21M2ua4QABre/eaQJu4CALlgJHh5KcsRKLi5V+rN8rX38y85rksMICC; AWSALBTGCORS=oauqL7VVR3ZhtBLHvQlk2f/FwFolND75LZgKcSh9mj81rbbCtKXarFgybh8vNAjL+B5bWVbv9v/oGOG8VXtoS7WK8BEtE6A1C0vXCTi8V/WetP6nz4Nss21M2ua4QABre/eaQJu4CALlgJHh5KcsRKLi5V+rN8rX38y85rksMICC; AWSALB=MWqJnU/SqtFJr3QDr+oQyjVxXWeGjflBQdSJ3pSi8Cr6ut9+VBiq7xG5VdoMS52WsKJPwz73cKtje8rMt6P2s8GpeucSsa0KvPy8yrlmpoz+zrISKbi5AldvhekX; _ga=GA1.2.245726608.1696600125; _ga_GZ9T4NRB7M=GS1.2.1696756177.1.1.1696756179.0.0.0'
}

In [3]:
def need_to_find_crisis(soup,cfg):
    
    
    def parse_relative_tag(cfg):
        relative_anchor_tag = cfg.relative_anchor_tag
        pattern = r'<(\w+)(.*?)(?:>(.*?)</\1>|/>)'
        match = re.match(pattern, relative_anchor_tag)
        
        if match:
            tag_name = match.group(1)
            attributes_str = match.group(2)
            text_content = match.group(3)
            
            
            attributes_pattern = r'(\w+)\s*=\s*["\'](.*?)["\']'
            attributes = dict(re.findall(attributes_pattern, attributes_str))
            
            return tag_name, attributes, text_content
        else:
            raise ValueError("Invalid relative_anchor_tag format")

    tag_name, attributes, text_content = parse_relative_tag(cfg)
    if soup.find(tag_name, attrs=attributes, text=text_content):
        return True
    return False


In [6]:
def spider(cfg):

    queue = deque([url for url in cfg.seed_url]) 
    visited_urls = set()
    result = []
    nums_urls = 1

    while len(result)<cfg.MAX_LENGTH:

        current_url= queue.popleft()
        
        try:
            response = requests.get(current_url,headers = headers)
        except (requests.exceptions.RequestException,
                UnicodeError):

            continue
        if not current_url.startswith(cfg.init_url):
            continue
        print(f'We have already visited {nums_urls} webpages, current url is:{current_url}')
        nums_urls += 1
        soup = BeautifulSoup(response.text, 'html.parser')

        links = soup.find_all('a',href=True)
        for link in links:
            relative_url = link.get('href')
            if 'http' not in relative_url:
                relative_url = cfg.init_url+relative_url

            if relative_url not in visited_urls:
                visited_urls.add(relative_url)
                queue.append(relative_url)
            
        
        if need_to_find_crisis(soup, cfg):
            if "crisis" in soup.get_text().lower():
                print(f"Press Release URL: {current_url}")
                result.append(current_url)
                with open(f'./{cfg.path}_{len(result)}.txt','w',encoding = 'utf-8') as f:
                    f.write(str(soup))



    with open(f'./urls_find_in_task_{cfg.path}.txt','w') as f:
        for url in result:
            f.write(url+'\n')

##### for United_Nations

In [7]:
cfg = configs.United_Nations
spider(cfg)

We have already visited 1 webpages, current url is:https://press.un.org/en
We have already visited 2 webpages, current url is:https://press.un.org#main-content
We have already visited 3 webpages, current url is:https://press.un.org/en
We have already visited 4 webpages, current url is:https://press.un.org/fr/home
We have already visited 5 webpages, current url is:https://press.un.org/en/content/secretary-general
We have already visited 6 webpages, current url is:https://press.un.org/en/content/secretary-general/press-release
We have already visited 7 webpages, current url is:https://press.un.org/en/content/secretary-general/press-conference
We have already visited 8 webpages, current url is:https://press.un.org/en/content/general-assembly
We have already visited 9 webpages, current url is:https://press.un.org/en/content/general-assembly/meetings-coverage
We have already visited 10 webpages, current url is:https://press.un.org/en/content/general-assembly/press-release
We have already vi

##### for European_Parliament

In [8]:
cfg = configs.European_Parliament
spider(cfg)

We have already visited 1 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/0
We have already visited 2 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/1
Press Release URL: https://www.europarl.europa.eu/news/en/press-room/page/1
We have already visited 3 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/2
Press Release URL: https://www.europarl.europa.eu/news/en/press-room/page/2
We have already visited 4 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/3
We have already visited 5 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/4
We have already visited 6 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/5
We have already visited 7 webpages, current url is:https://www.europarl.europa.eu/news/en/press-room/page/6
We have already visited 8 webpages, current url is:https://www.europarl.europa.eu/news/en/pr